In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy

In [2]:
def generate_random_graph(num_nodes, probability):
    graph = [[0] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < probability:
                graph[i][j] = 1
                graph[j][i] = 1

    return graph

def generate_random_solution(num_nodes):
    nodes = list(range(num_nodes))
    random.shuffle(nodes)
    return nodes

def calculate_total_edge_length(graph, solution):
    total_length = 0
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            if graph[i][j] == 1:
                position_i = solution.index(i)
                position_j = solution.index(j)
                total_length += abs(position_i - position_j)
    return total_length

# Example usage:
num_nodes = 10
edge_probability = 0.5

# Generate a random graph
graph = generate_random_graph(num_nodes, edge_probability)

# Generate a random solution
random_solution = generate_random_solution(num_nodes)

# Calculate the total edge length for the random solution
total_edge_length = calculate_total_edge_length(graph, random_solution)

print("Random Graph (Adjacency Matrix):")
for row in graph:
    print(row)

print("\nRandom Solution (Permutation):", random_solution)
print("Total Edge Length:", total_edge_length)

Random Graph (Adjacency Matrix):
[0, 0, 0, 1, 1, 1, 1, 0, 0, 1]
[0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 0, 0, 1, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 0, 1, 0]
[1, 1, 0, 1, 0, 1, 0, 0, 1, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

Random Solution (Permutation): [3, 9, 5, 7, 6, 4, 8, 2, 1, 0]
Total Edge Length: 83


In [3]:
import itertools

def brute_force(graph):
    num_nodes = len(graph)
    best_solution = None
    best_length = float('inf')

    # Generate all permutations of nodes
    all_permutations = itertools.permutations(range(num_nodes))

    # Check total edge length for each permutation
    for solution in all_permutations:
        total_length = calculate_total_edge_length(graph, solution)
        if total_length < best_length:
            best_length = total_length
            best_solution = solution

    return best_solution, best_length

In [4]:
brute_force(graph)

((2, 8, 5, 1, 6, 3, 0, 9, 4, 7), 36)

In [5]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [6]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [7]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [8]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [9]:
def local_search_first_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [10]:
def local_search_best_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [11]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
[3, 9, 5, 0, 6, 4, 8, 2, 1, 7] 67 1
[3, 9, 5, 0, 6, 4, 8, 2, 1, 7] 67 2
[3, 2, 5, 0, 6, 4, 8, 9, 1, 7] 65 3
[3, 2, 5, 0, 6, 4, 8, 9, 1, 7] 65 4
[3, 2, 5, 0, 6, 4, 8, 9, 1, 7] 65 5
[3, 2, 5, 0, 6, 4, 8, 9, 1, 7] 65 6
[3, 2, 5, 0, 6, 4, 8, 9, 1, 7] 65 7
[3, 2, 5, 0, 6, 4, 8, 9, 1, 7] 65 8
[3, 2, 4, 0, 6, 5, 8, 9, 1, 7] 64 9
[3, 2, 4, 0, 6, 5, 8, 9, 1, 7] 64 10
[3, 2, 4, 0, 8, 5, 6, 9, 1, 7] 62 11
[3, 2, 4, 0, 8, 5, 6, 9, 1, 7] 62 12
[3, 2, 4, 0, 8, 5, 6, 9, 1, 7] 62 13
[3, 2, 4, 0, 8, 5, 6, 9, 1, 7] 62 14
[3, 2, 4, 0, 8, 5, 6, 9, 1, 7] 62 15
[3, 2, 4, 0, 8, 5, 6, 9, 1, 7] 62 16
[4, 2, 3, 0, 8, 5, 6, 9, 1, 7] 60 17
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 18
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 19
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 20
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 21
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 22
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 23
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 24
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 25
[4, 2, 3, 0, 9, 5, 6, 8, 1, 7] 58 26
[4, 2, 3, 0

([7, 4, 9, 0, 3, 2, 6, 8, 5, 1], 40, 135)

In [12]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
[3, 1, 2, 8, 4, 6, 7, 5, 9, 0] 67 1
[3, 1, 2, 8, 4, 6, 7, 5, 9, 0] 67 2
[3, 2, 1, 8, 4, 6, 7, 5, 9, 0] 66 3
[3, 2, 1, 8, 4, 6, 7, 9, 5, 0] 65 4
[3, 2, 1, 8, 4, 6, 7, 9, 5, 0] 65 5
[3, 2, 1, 8, 7, 6, 4, 9, 5, 0] 63 6
[3, 2, 1, 8, 7, 6, 4, 9, 5, 0] 63 7
[3, 2, 1, 8, 7, 6, 4, 9, 5, 0] 63 8
[3, 2, 1, 8, 7, 6, 4, 9, 0, 5] 62 9
[3, 2, 1, 8, 7, 6, 4, 9, 0, 5] 62 10
[3, 2, 1, 8, 7, 6, 4, 9, 0, 5] 62 11
[3, 2, 1, 8, 7, 6, 4, 9, 0, 5] 62 12
[3, 2, 1, 8, 7, 6, 4, 9, 0, 5] 62 13
[3, 2, 1, 8, 7, 6, 4, 9, 0, 5] 62 14
[3, 2, 1, 8, 6, 7, 4, 9, 0, 5] 60 15
[3, 2, 1, 8, 6, 7, 4, 9, 0, 5] 60 16
[3, 2, 1, 8, 6, 7, 4, 9, 0, 5] 60 17
[3, 2, 1, 8, 6, 7, 4, 9, 0, 5] 60 18
[3, 2, 1, 8, 6, 7, 4, 9, 0, 5] 60 19
[3, 2, 1, 8, 6, 7, 9, 4, 0, 5] 58 20
[1, 2, 3, 8, 6, 7, 9, 4, 0, 5] 56 21
[1, 2, 3, 8, 6, 7, 9, 4, 0, 5] 56 22
[1, 2, 3, 8, 6, 7, 9, 4, 0, 5] 56 23
[1, 2, 3, 8, 6, 7, 9, 4, 0, 5] 56 24
[1, 2, 3, 8, 6, 7, 9, 4, 0, 5] 56 25
[1, 2, 3, 8, 6, 7, 9, 4, 0, 5] 56 26
[1, 2, 3, 8

([2, 8, 5, 1, 6, 3, 0, 9, 4, 7], 36, 205)

In [13]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i



In [14]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
[3, 5, 9, 7, 6, 8, 0, 1, 2, 4] 77 1
[3, 5, 9, 7, 6, 1, 0, 8, 2, 4] 73 2
[3, 5, 9, 7, 6, 1, 0, 4, 2, 8] 71 3
[3, 5, 9, 7, 6, 1, 0, 4, 2, 8] 71 4
[3, 1, 6, 7, 9, 5, 0, 4, 2, 8] 59 5
[3, 1, 6, 8, 2, 5, 0, 4, 7, 9] 54 6
[3, 1, 6, 8, 2, 5, 0, 4, 7, 9] 54 7
[3, 1, 6, 8, 2, 5, 0, 4, 7, 9] 54 8
[3, 1, 5, 2, 8, 6, 0, 4, 7, 9] 53 9
[3, 1, 5, 8, 2, 6, 0, 4, 9, 7] 51 10
[3, 1, 5, 8, 2, 6, 0, 4, 9, 7] 51 11
[3, 1, 5, 8, 2, 6, 0, 4, 9, 7] 51 12
[5, 1, 3, 8, 2, 6, 0, 4, 9, 7] 49 13
[7, 9, 4, 0, 6, 2, 8, 3, 5, 1] 48 14
[7, 9, 4, 0, 6, 2, 8, 3, 5, 1] 48 15
[7, 9, 4, 0, 6, 2, 8, 3, 5, 1] 48 16
[7, 4, 9, 0, 6, 2, 8, 3, 5, 1] 46 17
[7, 4, 9, 0, 6, 2, 3, 1, 5, 8] 44 18
[7, 4, 9, 0, 6, 2, 3, 1, 5, 8] 44 19
[7, 4, 9, 0, 6, 2, 3, 1, 5, 8] 44 20
[7, 4, 9, 0, 3, 1, 2, 6, 5, 8] 42 21
[7, 4, 9, 0, 3, 1, 2, 6, 5, 8] 42 22
[7, 4, 9, 3, 0, 1, 5, 6, 8, 2] 41 23
[7, 4, 9, 0, 3, 1, 5, 6, 8, 2] 39 24
[7, 4, 9, 0, 3, 1, 5, 6, 8, 2] 39 25
[7, 4, 9, 0, 3, 1, 5, 6, 8, 2] 39 26
[7, 4, 9, 0

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 76
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 77
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 78
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 79
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 80
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 81
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 82
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 83
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 84
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 85
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 86
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 87
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 88
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 89
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 90
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 91
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 92
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 93
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 94
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 95
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 96
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 97
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 98
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 99
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 100
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 101
[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] 36 10

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 43)

In [15]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
[3, 9, 5, 2, 6, 7, 0, 1, 4, 8] 79 1
[3, 9, 5, 8, 6, 7, 0, 1, 4, 2] 75 2
[3, 9, 6, 0, 1, 2, 4, 5, 7, 8] 63 3
[1, 9, 6, 0, 3, 2, 4, 5, 8, 7] 62 4
[1, 9, 6, 0, 3, 4, 2, 5, 8, 7] 61 5
[1, 9, 6, 8, 3, 5, 0, 2, 4, 7] 60 6
[1, 9, 6, 8, 3, 5, 0, 2, 4, 7] 60 7
[1, 9, 6, 8, 3, 5, 0, 2, 4, 7] 60 8
[1, 9, 6, 8, 3, 5, 0, 2, 4, 7] 60 9
[1, 9, 6, 8, 3, 5, 0, 2, 4, 7] 60 10
[1, 9, 6, 8, 5, 0, 3, 2, 4, 7] 59 11
[1, 9, 6, 8, 5, 0, 3, 2, 4, 7] 59 12
[1, 9, 6, 8, 5, 0, 3, 2, 4, 7] 59 13
[1, 9, 6, 8, 5, 0, 3, 2, 4, 7] 59 14
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 15
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 16
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 17
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 18
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 19
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 20
[1, 8, 6, 9, 5, 0, 3, 2, 4, 7] 55 21
[1, 5, 6, 9, 8, 0, 3, 2, 4, 7] 52 22
[1, 5, 6, 0, 8, 9, 3, 2, 4, 7] 48 23
[1, 5, 6, 0, 8, 9, 3, 2, 4, 7] 48 24
[1, 5, 6, 0, 8, 9, 3, 2, 4, 7] 48 25
[1, 5, 6, 0, 8, 9, 3, 2, 4, 7] 48 26
[8, 5, 6, 0

([2, 8, 5, 1, 6, 3, 0, 9, 4, 7], 36, 97)

In [16]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [17]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83
divs
[3, 9, 5, 4, 6, 7, 8, 2, 1, 0] 85
[3, 9, 2, 4, 6, 7, 8, 5, 1, 0] 71
[3, 9, 2, 4, 0, 7, 8, 5, 1, 6] 64
[4, 9, 2, 3, 0, 7, 8, 5, 1, 6] 60
[4, 7, 2, 3, 0, 9, 8, 5, 1, 6] 54
divs
[4, 6, 2, 3, 0, 9, 8, 5, 1, 7] 68
[8, 6, 2, 3, 0, 9, 4, 5, 1, 7] 56
[8, 6, 2, 3, 0, 9, 4, 5, 1, 7] 56
[8, 6, 2, 3, 0, 9, 1, 5, 4, 7] 54
[8, 6, 2, 3, 0, 9, 1, 5, 4, 7] 54
[8, 6, 2, 3, 0, 9, 1, 5, 4, 7] 54
[8, 6, 2, 3, 0, 9, 1, 5, 4, 7] 54
[8, 6, 2, 3, 5, 9, 1, 0, 4, 7] 53
[8, 6, 2, 3, 5, 9, 1, 0, 4, 7] 53
divs
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 7, 9, 1, 0, 4, 5] 63
[8, 6, 2, 3, 1, 9, 7, 0, 4, 5] 59
[8, 1, 2, 3, 6, 9, 7, 0, 4, 5] 56
[8, 1, 2, 3, 6, 9, 7, 0, 4, 5] 56
[8, 1, 2, 3, 6, 9, 7, 0, 4, 5] 56
[8, 1, 2, 3, 6, 9, 7, 0, 4, 5] 56
[8, 1, 2, 3, 6, 9, 7, 0, 4, 5] 56
[8, 1, 2, 3, 6, 9, 7, 0, 4, 5] 56

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 288)

In [18]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83
[6, 7, 5, 9, 3, 4, 8, 2, 1, 0] 81
[6, 7, 5, 9, 3, 4, 8, 2, 1, 0] 81
[6, 7, 5, 9, 3, 4, 8, 2, 1, 0] 81
[6, 7, 5, 9, 2, 8, 4, 3, 1, 0] 80
[6, 7, 5, 0, 1, 3, 4, 8, 2, 9] 74
divs
[6, 7, 5, 0, 2, 8, 4, 3, 1, 9] 80
[6, 7, 5, 0, 2, 8, 1, 3, 4, 9] 78
[3, 1, 8, 2, 0, 5, 7, 6, 4, 9] 66
[3, 1, 8, 2, 0, 5, 7, 6, 4, 9] 66
[1, 3, 8, 2, 0, 5, 7, 6, 4, 9] 64
[1, 3, 8, 2, 0, 5, 7, 6, 4, 9] 64
[1, 3, 8, 2, 0, 5, 7, 6, 4, 9] 64
[2, 8, 3, 1, 0, 5, 7, 6, 4, 9] 57
[2, 8, 3, 1, 0, 5, 7, 6, 4, 9] 57
[2, 8, 3, 1, 0, 5, 7, 6, 4, 9] 57
[2, 8, 3, 1, 0, 5, 7, 6, 4, 9] 57
divs
[2, 9, 4, 6, 7, 5, 0, 1, 3, 8] 71
[2, 9, 4, 6, 7, 5, 0, 1, 3, 8] 71
[2, 9, 4, 6, 7, 5, 0, 1, 3, 8] 71
[2, 9, 4, 6, 7, 5, 0, 1, 3, 8] 71
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 9, 3, 1, 0, 5, 7, 6, 4, 8] 65
[2, 

([2, 8, 5, 1, 6, 3, 0, 9, 4, 7], 36, 353)

In [19]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [20]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [21]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification    
            new_value = calculate_total_edge_length(graph, new_solution)
            print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)
            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i
                value = new_value
                solution = deepcopy(new_solution)
    return solution, value, best_i

In [22]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 5, 8, 6, 4, 7, 2, 1, 0] 84 0
[3, 9, 5, 8, 6, 4, 7, 2, 1, 0] 83 0
[3, 9, 5, 8, 6, 4, 0, 1, 2, 7] 74 1
[3, 8, 5, 9, 6, 4, 0, 1, 2, 7] 72 2
[3, 8, 5, 9, 6, 4, 0, 1, 2, 7] 72 3
[3, 8, 5, 9, 6, 4, 0, 1, 2, 7] 72 4
[3, 8, 5, 9, 6, 4, 0, 1, 2, 7] 72 5
[3, 8, 0, 4, 6, 9, 5, 1, 2, 7] 70 6
[3, 8, 0, 4, 6, 9, 5, 1, 2, 7] 70 7
[9, 6, 4, 0, 8, 3, 5, 1, 2, 7] 65 8
[9, 6, 4, 0, 8, 3, 5, 1, 2, 7] 65 9
vns:  [9, 0, 4, 6, 8, 3, 5, 1, 2, 7] 61 0
post shaking:  [9, 0, 1, 3, 8, 6, 5, 4, 2, 7] 67 0
[9, 0, 1, 3, 8, 6, 5, 4, 2, 7] 83 0
[9, 0, 1, 3, 6, 8, 5, 4, 2, 7] 62 1
[9, 3, 1, 0, 6, 8, 5, 4, 2, 7] 60 2
[9, 3, 1, 0, 6, 8, 5, 4, 2, 7] 60 3
[1, 3, 9, 0, 6, 8, 5, 4, 2, 7] 56 4
[1, 3, 9, 0, 6, 8, 5, 4, 2, 7] 56 5
[1, 3, 9, 0, 6, 8, 5, 4, 2, 7] 56 6
[1, 3, 9, 0, 6, 8, 5, 4, 2, 7] 56 7
[1, 3, 9, 0, 6, 8, 5, 4, 2, 7] 56 8
[1, 3, 9, 0, 6, 8, 5, 4, 2, 7] 56 9
vns:  [1, 3, 9, 0, 6, 4, 5, 8, 2, 7] 54 1
post shaking:  [1, 3, 9, 0, 4, 6, 5, 8, 2, 7] 55 1
[

([8, 2, 5, 1, 6, 3, 0, 9, 4, 7], 37, 89)

In [23]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 5, 7, 6, 4, 0, 1, 2, 8] 72 0
[3, 9, 5, 7, 6, 4, 0, 1, 2, 8] 83 0
[9, 3, 5, 7, 6, 4, 0, 1, 2, 8] 72 1
[9, 3, 5, 7, 6, 4, 0, 1, 2, 8] 72 2
[9, 3, 5, 7, 6, 4, 0, 1, 2, 8] 72 3
[1, 0, 4, 6, 7, 5, 3, 9, 2, 8] 65 4
[7, 6, 4, 0, 1, 5, 3, 9, 2, 8] 63 5
[7, 6, 4, 0, 9, 3, 5, 1, 2, 8] 54 6
[7, 6, 4, 0, 9, 3, 5, 1, 2, 8] 54 7
[7, 6, 4, 0, 9, 3, 5, 1, 2, 8] 54 8
[7, 6, 4, 0, 9, 3, 5, 1, 2, 8] 54 9
vns:  [7, 3, 9, 0, 4, 6, 5, 1, 2, 8] 50 0
post shaking:  [7, 6, 4, 0, 9, 3, 5, 1, 2, 8] 54 0
[7, 6, 4, 0, 9, 3, 5, 1, 2, 8] 83 0
[7, 6, 4, 0, 3, 9, 5, 1, 2, 8] 58 1
[7, 6, 4, 0, 3, 9, 5, 1, 2, 8] 58 2
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 3
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 4
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 5
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 6
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 7
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 8
[7, 0, 4, 6, 3, 9, 5, 1, 2, 8] 54 9
vns:  [7, 3, 9, 0, 4, 6, 5, 1, 2, 8] 50 0
post shaking:  [7, 3, 9, 0, 4, 8, 2, 1, 5, 6] 62 0
[

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 23)

In [24]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 5, 9, 7, 6, 4, 8, 2, 1, 0] 84 0
[3, 5, 9, 7, 6, 4, 8, 2, 1, 0] 83 0
[3, 5, 9, 7, 6, 4, 8, 2, 1, 0] 83 1
[3, 5, 9, 7, 6, 4, 8, 2, 1, 0] 83 2
[3, 4, 9, 7, 6, 5, 8, 2, 1, 0] 72 3
[3, 2, 9, 7, 6, 5, 8, 4, 1, 0] 70 4
[3, 2, 9, 7, 6, 5, 8, 4, 1, 0] 70 5
[3, 2, 8, 7, 6, 5, 9, 4, 1, 0] 66 6
[3, 2, 8, 7, 6, 5, 9, 4, 1, 0] 66 7
[3, 2, 8, 7, 6, 5, 9, 4, 1, 0] 66 8
[3, 2, 8, 7, 6, 5, 9, 4, 1, 0] 66 9
vns:  [3, 2, 8, 7, 6, 5, 9, 4, 1, 0] 66 0
post shaking:  [0, 2, 4, 7, 6, 5, 9, 8, 1, 3] 72 0
[0, 2, 4, 7, 6, 5, 9, 8, 1, 3] 83 0
[0, 2, 7, 4, 6, 5, 9, 8, 1, 3] 73 1
[0, 4, 7, 2, 6, 5, 9, 8, 1, 3] 69 2
[0, 4, 7, 3, 6, 5, 9, 8, 1, 2] 61 3
[0, 4, 7, 3, 6, 5, 9, 8, 1, 2] 61 4
[0, 4, 7, 3, 6, 5, 9, 8, 1, 2] 61 5
[0, 4, 7, 3, 6, 1, 9, 8, 5, 2] 60 6
[0, 4, 7, 3, 6, 1, 9, 8, 5, 2] 60 7
[0, 4, 7, 3, 6, 1, 9, 8, 5, 2] 60 8
[0, 4, 7, 5, 6, 1, 9, 8, 3, 2] 57 9
vns:  [0, 4, 7, 5, 6, 1, 9, 8, 3, 2] 57 1
post shaking:  [0, 4, 6, 5, 7, 1, 9, 8, 3, 2] 63 1
[

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 64)

In [25]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 5, 7, 6, 4, 8, 0, 1, 2] 77 0
[3, 9, 5, 7, 6, 4, 8, 0, 1, 2] 83 0
[3, 9, 8, 7, 6, 4, 5, 0, 1, 2] 73 1
[1, 9, 8, 7, 6, 4, 5, 0, 3, 2] 69 2
[4, 9, 8, 7, 6, 1, 5, 0, 3, 2] 59 3
[4, 9, 8, 7, 6, 1, 5, 0, 3, 2] 59 4
[4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 5
[4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 6
[4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 7
[4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 8
[4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 9
vns:  [4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 0
post shaking:  [4, 9, 0, 7, 6, 1, 2, 3, 8, 5] 55 0
[4, 9, 0, 7, 6, 1, 2, 3, 8, 5] 83 0
[8, 9, 0, 7, 6, 1, 2, 3, 4, 5] 79 1
[8, 9, 0, 7, 6, 1, 3, 2, 4, 5] 76 2
[8, 9, 0, 7, 6, 1, 3, 2, 4, 5] 76 3
[8, 9, 0, 7, 6, 1, 3, 2, 4, 5] 76 4
[8, 9, 0, 7, 6, 1, 3, 5, 4, 2] 72 5
[8, 9, 0, 7, 6, 1, 3, 5, 4, 2] 72 6
[8, 9, 0, 7, 6, 1, 3, 5, 4, 2] 72 7
[8, 2, 0, 7, 6, 1, 3, 5, 4, 9] 70 8
[8, 2, 0, 7, 6, 1, 3, 5, 4, 9] 70 9
vns:  [4, 9, 0, 7, 6, 1, 5, 8, 3, 2] 49 0
post shaking:  [4, 5, 1, 6, 7, 0, 9, 8, 3, 2] 60 0
[

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 34)

In [26]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 5, 7, 6, 8, 4, 2, 1, 0] 83 0
[3, 9, 5, 7, 6, 8, 4, 2, 1, 0] 83 0
[3, 9, 5, 7, 6, 8, 4, 2, 0, 1] 81 1
[3, 7, 5, 9, 6, 8, 4, 2, 0, 1] 79 2
[3, 7, 9, 5, 6, 8, 4, 2, 0, 1] 76 3
[3, 7, 9, 5, 6, 2, 8, 0, 1, 4] 74 4
[3, 7, 9, 5, 6, 2, 8, 0, 1, 4] 74 5
[3, 7, 9, 5, 6, 0, 8, 2, 1, 4] 70 6
[3, 7, 9, 5, 6, 0, 8, 2, 1, 4] 70 7
[3, 7, 9, 5, 6, 0, 8, 2, 1, 4] 70 8
[3, 7, 9, 4, 6, 0, 8, 2, 1, 5] 58 9
vns:  [3, 7, 9, 4, 0, 6, 8, 2, 1, 5] 55 0
post shaking:  [3, 6, 9, 0, 4, 7, 8, 2, 1, 5] 63 0
[3, 6, 9, 0, 4, 7, 8, 2, 1, 5] 83 0
[3, 6, 9, 0, 4, 8, 7, 2, 1, 5] 66 1
[3, 6, 9, 0, 4, 8, 7, 2, 1, 5] 66 2
[3, 6, 9, 0, 4, 8, 5, 2, 1, 7] 64 3
[3, 0, 9, 6, 4, 8, 5, 2, 1, 7] 62 4
[3, 0, 9, 6, 4, 8, 5, 2, 1, 7] 62 5
[3, 0, 9, 4, 6, 8, 5, 2, 1, 7] 61 6
[3, 0, 9, 4, 6, 8, 5, 2, 1, 7] 61 7
[3, 0, 9, 4, 6, 8, 5, 2, 1, 7] 61 8
[3, 0, 9, 4, 6, 8, 5, 2, 1, 7] 61 9
vns:  [3, 7, 9, 4, 0, 6, 8, 2, 1, 5] 55 1
post shaking:  [3, 7, 9, 4, 0, 6, 5, 2, 1, 8] 54 1
[

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 62)

In [27]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 5, 7, 2, 8, 4, 6, 1, 0] 83 0
[3, 9, 5, 7, 2, 8, 4, 6, 1, 0] 83 0
[3, 9, 6, 7, 2, 8, 4, 5, 1, 0] 73 1
[3, 9, 6, 7, 4, 8, 5, 0, 1, 2] 69 2
[3, 9, 7, 6, 4, 8, 5, 0, 1, 2] 67 3
[3, 9, 7, 6, 4, 8, 5, 0, 1, 2] 67 4
[3, 9, 7, 6, 4, 8, 5, 0, 1, 2] 67 5
[3, 9, 7, 6, 0, 8, 5, 4, 1, 2] 65 6
[3, 9, 7, 6, 0, 8, 5, 4, 1, 2] 65 7
[3, 9, 7, 6, 0, 8, 5, 4, 1, 2] 65 8
[3, 9, 7, 6, 0, 8, 4, 5, 1, 2] 64 9
vns:  [4, 9, 7, 6, 0, 8, 3, 5, 1, 2] 50 0
post shaking:  [4, 9, 7, 6, 0, 8, 2, 1, 5, 3] 58 0
[4, 9, 7, 6, 0, 8, 2, 1, 5, 3] 83 0
[1, 9, 7, 6, 0, 8, 2, 4, 5, 3] 78 1
[1, 9, 3, 6, 0, 8, 2, 4, 5, 7] 61 2
[1, 9, 3, 6, 0, 8, 2, 4, 5, 7] 61 3
[1, 9, 3, 6, 0, 8, 2, 4, 5, 7] 61 4
[1, 9, 3, 6, 0, 8, 2, 5, 4, 7] 58 5
[1, 9, 3, 6, 0, 8, 2, 5, 4, 7] 58 6
[1, 9, 3, 6, 0, 8, 2, 5, 4, 7] 58 7
[1, 9, 3, 6, 5, 8, 2, 0, 4, 7] 57 8
[1, 9, 3, 6, 5, 8, 2, 0, 4, 7] 57 9
vns:  [4, 9, 7, 6, 0, 8, 3, 5, 1, 2] 50 0
post shaking:  [4, 9, 7, 6, 0, 2, 1, 5, 3, 8] 58 0
[

([7, 4, 9, 0, 3, 6, 1, 5, 8, 2], 36, 5)

In [28]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 6, 7, 5, 4, 8, 2, 1, 0] 79 0
[3, 9, 6, 7, 5, 4, 8, 2, 1, 0] 83 0
[3, 8, 4, 5, 7, 9, 0, 1, 6, 2] 81 1
[3, 8, 4, 5, 9, 7, 0, 6, 1, 2] 79 2
[3, 8, 4, 5, 1, 6, 0, 7, 9, 2] 75 3
[3, 8, 4, 5, 1, 6, 0, 7, 9, 2] 75 4
[5, 4, 8, 3, 1, 6, 0, 7, 9, 2] 70 5
[5, 9, 7, 0, 6, 1, 3, 8, 4, 2] 64 6
[5, 9, 7, 0, 6, 1, 3, 8, 4, 2] 64 7
[5, 9, 7, 0, 6, 1, 3, 8, 4, 2] 64 8
[5, 9, 7, 0, 6, 1, 3, 8, 4, 2] 64 9
vns:  [5, 9, 7, 0, 6, 4, 8, 3, 1, 2] 62 0
post shaking:  [6, 9, 7, 0, 5, 4, 3, 8, 1, 2] 64 0
[6, 9, 7, 0, 5, 4, 3, 8, 1, 2] 83 0
[2, 1, 8, 3, 4, 5, 0, 7, 9, 6] 64 1
[2, 1, 8, 3, 4, 5, 0, 7, 9, 6] 64 2
[2, 1, 8, 3, 5, 4, 0, 7, 9, 6] 61 3
[2, 1, 8, 3, 5, 6, 9, 7, 0, 4] 47 4
[2, 1, 8, 3, 5, 6, 9, 0, 7, 4] 44 5
[2, 1, 8, 3, 5, 6, 9, 0, 7, 4] 44 6
[2, 1, 8, 3, 5, 6, 9, 0, 7, 4] 44 7
[2, 1, 8, 3, 5, 6, 9, 0, 7, 4] 44 8
[2, 1, 8, 3, 5, 6, 9, 0, 7, 4] 44 9
vns:  [2, 1, 8, 3, 5, 6, 9, 0, 7, 4] 44 1
post shaking:  [2, 1, 4, 3, 5, 6, 9, 0, 7, 8] 64 1
[

([2, 8, 5, 1, 6, 3, 0, 9, 4, 7], 36, 48)

In [29]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
post shaking:  [3, 9, 5, 7, 6, 4, 8, 0, 1, 2] 77 0
[3, 9, 5, 7, 6, 4, 8, 0, 1, 2] 83 0
[3, 5, 9, 7, 6, 4, 8, 0, 1, 2] 78 1
[3, 5, 9, 7, 6, 4, 8, 0, 1, 2] 78 2
[3, 2, 1, 0, 8, 4, 6, 7, 9, 5] 68 3
[3, 2, 1, 0, 8, 4, 6, 7, 9, 5] 68 4
[3, 2, 1, 0, 8, 4, 6, 9, 5, 7] 63 5
[3, 2, 1, 0, 8, 6, 4, 9, 5, 7] 60 6
[3, 2, 1, 0, 8, 6, 4, 9, 5, 7] 60 7
[3, 2, 1, 0, 8, 6, 4, 5, 9, 7] 59 8
[3, 2, 1, 0, 8, 6, 4, 5, 9, 7] 59 9
vns:  [3, 2, 1, 0, 8, 6, 4, 5, 9, 7] 59 0
post shaking:  [3, 2, 1, 5, 4, 6, 8, 0, 9, 7] 61 0
[3, 2, 1, 5, 4, 6, 8, 0, 9, 7] 83 0
[4, 5, 1, 2, 3, 6, 8, 0, 9, 7] 65 1
[4, 5, 1, 2, 3, 6, 8, 0, 9, 7] 65 2
[4, 5, 1, 2, 3, 6, 8, 0, 9, 7] 65 3
[4, 5, 1, 2, 3, 6, 8, 0, 9, 7] 65 4
[4, 5, 1, 2, 3, 6, 8, 0, 9, 7] 65 5
[4, 5, 1, 2, 3, 6, 8, 0, 9, 7] 65 6
[4, 5, 1, 2, 3, 8, 6, 0, 9, 7] 64 7
[4, 7, 9, 0, 6, 8, 3, 2, 1, 5] 46 8
[4, 7, 9, 0, 6, 5, 1, 3, 2, 8] 41 9
vns:  [7, 4, 9, 0, 6, 5, 1, 3, 2, 8] 40 0
post shaking:  [7, 4, 9, 2, 3, 1, 5, 6, 0, 8] 51 0
[

([7, 4, 9, 0, 3, 6, 5, 1, 8, 2], 37, 7)

In [30]:
#genetic algorithms

In [31]:
class Individual:
    def __init__(self, num_nodes, graph):
        self.num_nodes = num_nodes
        self.graph = graph
        self.code = self.generate_solution(self.num_nodes)
        self.fitness = self.calc_fitness(self.num_nodes, self.graph)
    
    def generate_solution(self, num_nodes):
        nodes = list(range(num_nodes))
        random.shuffle(nodes)
        return nodes

    def calc_fitness(self, num_nodes, graph):
        total_length = 0
        for i in range(len(graph)):
            for j in range(i + 1, len(graph)):
                if graph[i][j] == 1:
                    position_i = self.code.index(i)
                    position_j = self.code.index(j)
                    total_length += abs(position_i - position_j)
        return -total_length

In [32]:
i = Individual(10, graph)

In [33]:
print(i.code)
print(i.fitness)

[5, 2, 7, 3, 0, 9, 4, 6, 1, 8]
-69


In [34]:
def tournament_selection(population, tour_size, forbidden):
    tournament = random.sample(range(len(population)), tour_size)
    #selected = max(tournament, key=lambda x: x.fitness)
    best_i = -1
    best_fitness = population[tournament[0]].fitness
    for i in tournament:
        #print('fitness:', population[i].fitness)
        if population[i].fitness >= best_fitness and i != forbidden:
            best_fitness = population[i].fitness
            best_i = i

    #print('best_fitness:', best_fitness)
    return best_i

In [35]:
# its like a wheel in a casino but instead of having equal proportions to
# for wheels slots we have different proportions(probabilities) for each slot
# its size depends on the fitness of an individual
# sum(i from 0 to n)pi where pi is probability of each one
def roulette_selection(population):
    total_fitness = sum(ind.fitness for ind in population)
    fitness_props = [ind.fitness / total_fitness for ind in population]
    cumulative_props = [sum(fitness_props[:i+1] for i in range(len(fitness_props)))]
    
    rand_prop = random.uniform(0,1)
    for i, prob in enumerate(cumulative_props):
        if rand_prop < prob:
            return population[i]

In [36]:
#beacause we are dealing with permutations, we have to be careful here and used sepcialized crossover algorithms
#ordered crossover method
def ordered_crossover(parent1, parent2, cx_point1 = -1, cx_point2 = -1):
    
    n = len(parent1.code)

    # Choose two random crossover points
    if cx_point1 == -1 and cx_point2 == -1:
        cx_point1, cx_point2 = sorted(random.sample(range(n), 2))
        #print(cx_point1, cx_point2)

    # Copy the segment between the crossover points from parent1 to child1
    child1_segment = parent1.code[cx_point1:cx_point2 + 1]

    # Fill the remaining positions in child1 with elements from parent2
    child1 = [-1] * n
    child1[cx_point1:cx_point2 + 1] = child1_segment
    remaining_positions = [i for i in parent2.code if i not in child1_segment]
    j = 0
    for i in range(n):
        if child1[i] == -1:
            child1[i] = remaining_positions[j]
            j += 1

    return child1, cx_point1, cx_point2


In [37]:
def pmx_crossover(parent1, parent2):
    n = len(parent1.code)
    start, end = sorted(random.sample(range(n), 2))
    
    child = [-1] * n
    child[start:end + 1] = parent1.code[start:end + 1]

    for i in range(start, end + 1):
        if parent2.code[i] not in child:
            #print(parent1.code)
            #print(parent2.code)
            index = parent2.code.index(parent1.code[i])
            #this next lines are the only difference between pmx and ordered
            #we dont change child1 in a ordered way but according to previous indexes
            while child[index] != -1:
                index = parent2.code.index(parent1.code[index])
            child[index] = parent2.code[i]
            
            
    for i in range(n):
        if child[i] == -1:
            child[i] = parent2.code[i]

    return child

In [38]:
#with linear ranking selection is not dependable on fitness but fitness rank
#this way worse individuals can get a chance to be choosen and maybe they can 
# give better results
def rank_selection_linear_ranking(population, num_selections,forbidden,selection_pressure = 1.5):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [(2-selection_pressure)/n + 2*i*(selection_pressure-1)/(n*(n-1)) for i in range(n)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)
    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [39]:

def rank_selection(population, num_selections, forbidden):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [i/n for i in range(1, n + 1)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)

    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [40]:
def mutation_swap(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            rand_i = random.choice(range(len(individual.code)))
            tmp = individual.code[i]
            individual.code[i] = individual.code[rand_i]
            individual.code[rand_i] = tmp

In [41]:
def mutation_inverse(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            index1, index2 = sorted(random.sample(range(len(individual.code)), 2))
            individual.code[index1:index2+1] = reversed(individual.code[index1:index2+1])

In [42]:
def genetic_algo(population_size, graph, num_nodes,num_generations,tournament_size, mutation, mutation_prob, elitism_size, crossover, selection, ordered_cross = False):
    #if use_elitism and (population_size - elitism_size) % 2 == 1:
    #    elitism_size += 1
    
    population = [Individual(num_nodes, graph) for _ in range(population_size)]
    new_population = population.copy()
    
    for i in range(num_generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1_i = selection(population, tournament_size, forbidden=-2)
            parent2_i = selection(population, tournament_size, parent1_i)
            #disabled - causes bloated file size
            #print('gen:', i, 'iter:', j, 'parents:', parent1_i, parent2_i)
            if ordered_cross:
                new_population[j].code, i1, i2 = crossover(population[parent1_i], population[parent2_i])
                new_population[j+1].code, _, _ = crossover(population[parent2_i], population[parent1_i], i1, i2)
            else:    
                new_population[j].code = crossover(population[parent1_i], population[parent2_i])
                new_population[j+1].code = crossover(population[parent1_i], population[parent2_i])
        
            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness(num_nodes, graph)
            new_population[j+1].fitness = new_population[j+1].calc_fitness(num_nodes, graph)
            
        population = new_population.copy()
    return max(population, key=lambda x: x.fitness)

In [43]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[2, 8, 5, 1, 6, 3, 0, 9, 4, 7] -36


In [44]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [45]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=20,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=tournament_selection)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [46]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4, mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=tournament_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [47]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [48]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection_linear_ranking, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[2, 8, 5, 1, 6, 3, 0, 9, 4, 7] -36


In [49]:
#same algorithms with mutations that inverse a part of the solution instead of performing swaps
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=10, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [50]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [51]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=20,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=tournament_selection)
print(best_individual.code, best_individual.fitness)

[2, 8, 5, 1, 6, 3, 0, 9, 4, 7] -36


In [52]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4, mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=tournament_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [53]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 6, 1, 5, 8, 2] -36


In [54]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection_linear_ranking, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[7, 4, 9, 0, 3, 2, 6, 8, 5, 1] -40


In [55]:
#linear programming

In [56]:
from docplex.mp.model import Model

In [57]:
def linear_integer(graph):
    num_nodes = len(graph)

    # Create a CPLEX model
    model = Model(name='minimum_linear_arrangement')

    # Create integer decision variables for the positions (1D array)
    positions = model.integer_var_list(num_nodes, name='position')

    model.add_constraints(positions[i] <= (num_nodes - 1) for i in range(num_nodes))
    model.add_constraints(positions[i] >= 0 for i in range(num_nodes))

    # Constraints: Each node must appear exactly once in the permutation
    model.add_constraints(model.sum((positions[i] == j) for j in range(num_nodes)) == 1 for i in range(num_nodes))

    # Constraints: Avoid subcycles (no repeated nodes)
    model.add_constraints(positions[i] != positions[j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    # Example: Prioritize edges between adjacent nodes
    model.minimize(model.sum(graph[i][j] * ((positions[i] - positions[j])**2) for i in range(num_nodes) for j in range(num_nodes)))


    # Solve the model
    model.solve()

    # Get the solution
    mla_positions = [positions[i].solution_value for i in range(num_nodes)]

    return mla_positions

In [58]:
li_soulution = linear_integer(graph)
#print(li_soulution)
li_soulution = list(map(lambda x: int(x), li_soulution))
print(li_soulution, calculate_total_edge_length(graph, li_soulution))

[3, 6, 9, 4, 1, 7, 5, 0, 8, 2] 66


In [59]:
def linear_integer_squared(graph):
    num_nodes = len(graph)

    # Create a CPLEX model
    model = Model(name='minimum_linear_arrangement')

    # Create integer decision variables for the positions (1D array)
    positions = model.integer_var_list(num_nodes, name='position')

    model.add_constraints(positions[i] <= (num_nodes - 1) for i in range(num_nodes))
    model.add_constraints(positions[i] >= 0 for i in range(num_nodes))

    # Constraints: Each node must appear exactly once in the permutation
    model.add_constraints(model.sum((positions[i] == j) for j in range(num_nodes)) == 1 for i in range(num_nodes))

    # Constraints: Avoid subcycles (no repeated nodes)
    model.add_constraints(positions[i] != positions[j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    # Objective: Minimize the total edge length
    model.minimize(model.sum(graph[i][j] * model.abs(positions[i] - positions[j]) for i in range(num_nodes) for j in range(num_nodes)))

    # Solve the model
    model.solve()

    # Get the solution
    mla_positions = [positions[i].solution_value for i in range(num_nodes)]

    return mla_positions

In [60]:
li_soulution = linear_integer_squared(graph)
li_soulution = list(map(lambda x: int(x), li_soulution))
print(li_soulution, calculate_total_edge_length(graph, li_soulution))

[3, 6, 9, 4, 1, 7, 5, 0, 8, 2] 66


In [61]:
#this method uses too many variables for the community version of CPLEX
def linear_integer_with_preprocessing(graph):
    num_nodes = len(graph)

    # Create a CPLEX model
    model = Model(name='minimum_linear_arrangement')

    initial_solution, initial_value, _ = local_search(graph, random_solution, total_edge_length, 10, change_func=make_change_swap)

    # Create integer decision variables for the positions (1D array)
    positions = model.integer_var_list(num_nodes, name='position')

    model.add_constraints(positions[i] <= (num_nodes - 1) for i in range(num_nodes))
    model.add_constraints(positions[i] >= 0 for i in range(num_nodes))

    # Constraints: Each node must appear exactly once in the permutation
    model.add_constraints(model.sum((positions[i] == j) for j in range(num_nodes)) == 1 for i in range(num_nodes))

    # Constraints: Avoid subcycles (no repeated nodes)
    model.add_constraints(positions[i] != positions[j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    model.add_constraint(model.sum(graph[i][j] * model.abs(positions[i] - positions[j]) for i in range(num_nodes) for j in range(num_nodes)) <= initial_value)

    # Objective: Minimize the total edge length
    model.minimize(model.sum(graph[i][j] * model.abs(positions[i] - positions[j]) for i in range(num_nodes) for j in range(num_nodes)))

    # Solve the model
    model.solve()

    # Get the solution
    mla_positions = [positions[i].solution_value for i in range(num_nodes)]

    return mla_positions

In [63]:
li_soulution = linear_integer_with_preprocessing(graph)
li_soulution = list(map(lambda x: int(x), li_soulution))
print(li_soulution, calculate_total_edge_length(graph, li_soulution))

[3, 9, 5, 7, 6, 4, 8, 2, 1, 0] 83 0
[3, 9, 2, 7, 6, 4, 8, 5, 1, 0] 69 1
[3, 9, 2, 7, 8, 4, 6, 5, 1, 0] 67 2
[3, 9, 2, 7, 8, 4, 6, 5, 1, 0] 67 3
[3, 9, 2, 7, 8, 4, 6, 5, 1, 0] 67 4
[3, 9, 2, 7, 8, 4, 6, 0, 1, 5] 63 5
[3, 9, 2, 7, 8, 4, 6, 0, 1, 5] 63 6
[3, 9, 2, 7, 8, 4, 6, 0, 1, 5] 63 7
[3, 9, 2, 7, 8, 4, 6, 0, 1, 5] 63 8
[3, 9, 2, 7, 8, 4, 6, 0, 1, 5] 63 9


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 1000 vars, 911 consts, CPLEX code=1016

In [64]:
#greedy algorigram

In [94]:
#vidi da li moze bolje
def greedy_linear_arrangement(graph, start_node):
    num_nodes = len(graph)
    #linear_arrangement = [random.choice(range(len(graph)))]
    linear_arrangement = [start_node]

    # Greedy algorithm
    for _ in range(num_nodes - 1):
        best_node = None
        best_cost = float('inf')

        for node in range(num_nodes):
            if node not in linear_arrangement:
                current_cost = 0
                for placed_node in linear_arrangement:
                    current_cost += graph[node][placed_node]

                if current_cost < best_cost:
                    best_cost = current_cost
                    best_node = node

        linear_arrangement.append(best_node)

    return linear_arrangement

In [99]:
greedy_solution = greedy_linear_arrangement(graph, 7)
greedy_value = calculate_total_edge_length(graph, greedy_solution)
print(greedy_solution, greedy_value)

[7, 0, 1, 2, 8, 4, 3, 5, 9, 6] 82


In [100]:
greedy_solutions = [None for i in range(num_nodes)]
greedy_values = [None for i in range(num_nodes)]
best_greedy_value = float('inf')
best_greedy_i = 0
for i in range(num_nodes):
    greedy_solutions[i] = greedy_linear_arrangement(graph, i)
    greedy_values[i] = calculate_total_edge_length(graph, greedy_solutions[i])

    if greedy_values[i] < best_greedy_value:
        best_greedy_value = greedy_values[i]
        best_greedy_i = i

print(greedy_solutions[best_greedy_i], best_greedy_value)    

[5, 2, 4, 1, 7, 0, 8, 3, 9, 6] 75
